<a href="https://colab.research.google.com/github/eric88525/Tbrain_NLP_game/blob/master/QAFilter_package.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QA filter

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
!pip install transformers
!pip install opencc


     |████████████████████████████████| 778kB 8.0MB/s 
     |████████████████████████████████| 3.0MB 29.9MB/s 
     |████████████████████████████████| 890kB 39.6MB/s 
     |████████████████████████████████| 1.1MB 54.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b70d127ec19e7d1753058b17ae646c310584a5fd27b6dd2a7bc81001eeeab73b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.3MB 9.4MB/s 


In [ ]:
from opencc import OpenCC
from transformers import *
import torch
import torch.nn as nn
import pandas as pd
import csv
import ast

In [ ]:
cd ./drive/My Drive/Colab Notebooks/TBrain/nosep

/content/drive/My Drive/Colab Notebooks/TBrain/nosep


In [ ]:
data = pd.read_csv('2020-07-30.csv')
# article binary ckip_name predict_name

In [ ]:
len(data)

375

In [ ]:
class bertwwmQA(nn.Module):
    def __init__(self,model_name,config):
        super(bertwwmQA,self).__init__()
        #self.bert_model = BertModel.from_pretrained("hfl/chinese-roberta-wwm-ext-large")
        
        self.bert_model = BertModel.from_pretrained(model_name,config = config)
        self.bi_decoder = nn.Sequential(
            nn.Linear(config.hidden_size,config.hidden_size)
            ,nn.Dropout(0.1)
            ,nn.ReLU()
            ,nn.Linear(config.hidden_size,2)
        )
        #self.start()

    def start(self):
      nn.init.xavier_uniform_(self.bi_decoder[0].weight)
      nn.init.constant_(self.bi_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.bi_decoder[3].weight)
      nn.init.constant_(self.bi_decoder[3].bias, 0)

    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None):
       # print(f'receive input_ids {input_ids}')

        cls = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[1]
        binary = self.bi_decoder(cls)
        return binary

In [ ]:
class QAFilter():
  def __init__(self,model_path): 
    
    # device
    self.device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    
    # model & tokenizer
    model_type = 'hfl/chinese-roberta-wwm-ext'
    config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
    self.tokenizer = BertTokenizer.from_pretrained(model_type)
    self.model = bertwwmQA(model_type,config).to(self.device)
    self.model.load_state_dict(torch.load(model_path)) 
  
    # 繁簡轉換
    self.c2tw = OpenCC('s2t') # china to tw
    self.tw2c = OpenCC('t2s') # tw to china

  def filter(self,content,name_list):
    # name_list : a list like: ['蔡英文','陳水扁']
    # content  : a new "繁體新聞"
    # 會回傳像是 [1,0,0,1] 代表第0和3的人名是對的
    content = self.tw2c.convert(content)
    pred_nlist = []
    with torch.no_grad():
      for n in name_list:
        nc = self.tw2c.convert(n)
        token_tensor = self.tokenizer.encode_plus(str(nc),str(content),max_length=512,truncation=True,pad_to_max_length=True)
        token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(self.device)
        segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(self.device)
        mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(self.device)
        pred = self.model(input_ids=token,attention_mask=mask,token_type_ids=segment).argmax(dim=1)
        #print(n,pred.item())
        if pred.item() == 1:
          pred_nlist.append(n)
    return pred_nlist

In [ ]:
mf = QAFilter('./saved_models/QA_Filter.pt')

In [ ]:
# article binary ckip_name predict_name
with open('./compare0730.csv','w',encoding="utf-8",newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['idx','article','binary', 'ckip_name' , 'predict_name' , 'myQA_plus_ckip','比pred少抓的','比pred多抓的'])
  for i in range(len(data)):
    if data['binary'][i] == 1:
      ckips = ast.literal_eval(data['ckip_name'][i])
      ori_pred = ast.literal_eval(data['predict_name'][i])
      pred = mf.filter(data['article'][i],ckips)
      writer.writerow([ i+2 , data['article'][i] , data['binary'][i], data['ckip_name'][i] \
              , ori_pred ,pred ,set(ori_pred)-set(pred) ,set(pred)-set(ori_pred) ]) 

In [ ]:
content = "〔財經頻道／綜合報導〕2013年爆發的台苯掏空弊案，至今尚在漫長的司法訴訟之中，台苯大股東何山一審遭地方法院依三個背信罪判3年8個月有期徒刑，網友稱這是對台灣上市公司治理最大的諷刺，經司法判決掏空公司者，至今仍主導台苯經營權，國家的律法規定對這種不合理現象束手無策。何山掏空台苯案，近日高等法院將開庭審理。何至今仍是台苯董事，為最大單一股東美好實業的法人代表。根據台北地檢署偵辦台苯掏空案，查出全案可分為4大區塊包括，天籟大飯店股權交易收佣金、致恩公司財報不實、違法貸款及投資景碩公司，以及違背任務損及美都公司利益等；而涉入本案的台苯董事長、同時是經濟部前常務次長張鈞傑與天籟集團董事長何山等人，則被依法起訴。案經台北地院審理，2018年6月依虛偽記載財務報告罪判張3年刑期，何依3個背信罪定應執行刑3年8月、不法所得631萬5600元全數沒收。台北地檢承辦本案過程，針對天籟大飯店股權交易部分，認定何山、何的親信劉結依、蘇一為與陳敏盛等4人，利用台苯旗下力福工程等6家子公司進行天籟股權交易時，透過人頭、協助何山取得佣金2664萬8000元。何山在地檢署承認收回扣但不承認掏空，表示收回扣是買賣市場常規，沒有犯法。台苯預計在108年1月31日舉行股東臨時會，將提前改選董監事，董事會也將由9席董事、3席監察人，調整為11席董事，原本屬於何山陣營的民進黨大老吳重賢子女吳恆及吳永，都已經表明不再參與，使得這場董監改選未演先轟動。"
ckip = ['蘇一為', '吳永，', '陳敏盛', '吳恆', '張鈞傑', '吳重賢', '劉結依', '何山']
print(mf.filter(content,ckip))